In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Transactions')

+-----+---------+----------+--------+------------+
|  id | country |  state   | amount | trans_date |
+-----+---------+----------+--------+------------+
| 121 |    US   | approved |  1000  | 2018-12-18 |
| 122 |    US   | declined |  2000  | 2018-12-19 |
| 123 |    US   | approved |  2000  | 2019-01-01 |
| 124 |    DE   | approved |  2000  | 2019-01-07 |
+-----+---------+----------+--------+------------+


### Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

```
Output: 
+----------+---------+-------------+----------------+--------------------+-----------------------+
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
+----------+---------+-------------+----------------+--------------------+-----------------------+
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |
+----------+---------+-------------+----------------+--------------------+-----------------------+
```


In [5]:
%%sql 

SELECT 
    SUBSTRING(trans_date, 1, 7) AS month 
FROM Transactions;

month
2018-12
2018-12
2019-01
2019-01


In [14]:
%%sql 

SELECT 
    SUBSTRING(trans_date, 1, 7) AS month,
    country,
    COUNT(*) AS trans_count,
    SUM(amount) AS trans_total_amount
FROM Transactions
GROUP BY country, month

month,country,trans_count,trans_total_amount
2018-12,US,2,3000
2019-01,US,1,2000
2019-01,DE,1,2000


In [18]:
%%sql 

SELECT 
    SUBSTRING(trans_date, 1, 7) AS month,
    country,
    COUNT(*) AS trans_count,
    SUM(CASE WHEN state = 'approved' THEN 1 ELSE 0 END) AS approved_count,
    SUM(amount) AS trans_total_amount,
    SUM(CASE WHEN state = 'approved' THEN amount ELSE 0 END) AS approved_total_amount
FROM Transactions
GROUP BY country, month;

month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
2018-12,US,2,1,3000,1000
2019-01,US,1,1,2000,2000
2019-01,DE,1,1,2000,2000


## Using CTE

In [37]:
%%sql 

WITH monthly_stats AS (
    SELECT
        SUBSTRING(trans_date, 1, 7) AS month,
        country,
        COUNT(*) AS trans_count,
        SUM(amount) AS trans_total_amount
    FROM Transactions
    GROUP BY month, country
),
approved_stats AS (
    SELECT
        SUBSTRING(trans_date, 1, 7) AS month,
        country,
        COUNT(*) AS approved_count,
        SUM(amount) AS approved_total_amount
    FROM Transactions
    WHERE state = 'approved'
    GROUP BY month, country
)
SELECT
    monthly_stats.month,
    monthly_stats.country,
    monthly_stats.trans_count,
    approved_stats.approved_count,
    monthly_stats.trans_total_amount,
    approved_stats.approved_total_amount
FROM monthly_stats
JOIN approved_stats ON monthly_stats.month = approved_stats.month
    AND monthly_stats.country = approved_stats.country;


month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
2018-12,US,2,1,3000,1000
2019-01,US,1,1,2000,2000
2019-01,DE,1,1,2000,2000
